In [3]:
using DataFrames, ForwardDiff

In [4]:
df = readtable("model_australia.txt", separator = ' ', header = false)

names!(df, [Symbol("C$i") for i in 1:4])

head(df)

,C1,C2,C3,C4
1,1,0,0,0
2,0,1,0,0
3,0,0,1,0
4,35,0,0,0
5,69,34,35,0
6,70,71,70,30


In [5]:
function f(β::Vector, model::Float64 = 0.0, n::Int64 = 210)
    i = 1
    while i <= n
        probability = individual(β, i)
        model += log(probability(β))
        i += 1
    end
    return model/n
end

f (generic function with 3 methods)

In [6]:
# f_ind

function individual(β::Vector, i::Int64)
    
    ## Les trois premières lignes correspondent aux ASC, et devraient donc être utilisées.
    
    data = convert(Array, df[i*7-3:i*7-1, :])
    choices = convert(Array, df[i*7:i*7, :])
    alternatives = find(choices .== 0)
    choice = find(choices .== 1)[1]
    
    # Utilité simple.
    function utility(β::Vector, i::Int64)
        return dot(vec(data[:, i]), β)
    end
    
    # Tel que proposé à notre dernière rencontre. Somme des différences des utilités.
    function probability(β::Vector, t::Float64 = 0.0)
        ## We should avoid to recompute the utility of the chosen alternative every time.
        c = utility(β, choice)
        for alternative in alternatives
            t += exp(utility(β, alternative)-c)
        end
        return 1/(1+t)
    end
    
    return probability
end

individual (generic function with 1 method)

In [7]:
function g(β::Vector)
    return ForwardDiff.gradient(f, β)
end

g (generic function with 1 method)

In [8]:
# Calcul du gradient par individu.

function g_test(β::Vector, t::Float64 = 0.0, n::Int64 = 210)
    for i = 1:n
        probability = individual(β, i)
        t += (1/probability(β))*ForwardDiff.gradient(probability, β)
    end
    
    # Pourquoi t/n alors que dérivée d'une somme = somme des dérivées?
    ## Parce qu'à la fin du calcul de f, on applique le facteur 1/n.
    return t/n
end

g(zeros(3)) - g_test(zeros(3))

3-element Array{Float64,1}:
 0.0
 0.0
 0.0

In [9]:
g(zeros(3))

3-element Array{Float64,1}:
  2.8869 
 -9.57976
 -7.05595

In [10]:
g_test(zeros(3))

3-element Array{Float64,1}:
  2.8869 
 -9.57976
 -7.05595

In [11]:
function g!(β::Vector, storage::Vector)
    s = g_test(β)
    storage[1:length(s)] = s[1:length(s)]
end

g! (generic function with 1 method)

In [12]:
function H(β::Vector)
    return ForwardDiff.hessian(f, β)
end

H(zeros(3))

3×3 Array{Float64,2}:
 -296.292   -213.639    92.0964
 -213.639   -589.446  -122.245 
   92.0964  -122.245  -517.648 

In [13]:
function H!(β::Vector, storage::Matrix)
    s = H(β)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

H! (generic function with 1 method)

In [14]:
function newton(f::Function, g::Function, h::Function, β0::Vector,
        k::Int64 = 1, δ::Float64 = 1e-6, nmax::Int64 = 1000)
    β = β0
    n = length(β)
    δ *= δ
    H = eye(n)
    dfβ = ones(n)
    g(β, dfβ)
    while dot(dfβ, dfβ) > δ && k <= nmax
        g(β, dfβ)
        h(β, H)
        β -= H\dfβ
        k += 1
    end
    β, k
end

newton (generic function with 4 methods)

In [15]:
newton(f, g!, H!, zeros(3))

([0.0283255, -0.0257532, -0.00362244], 6)